In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())
import os   

In [ ]:
from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-crewai-app", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

In [ ]:

os.environ["OPENAI_API_KEY"] = os.environ["GROQ_API_KEY"] 
os.environ["OPENAI_API_BASE"] = os.environ["GROQ_API_BASE"] 
os.environ["OPENAI_MODEL_NAME"] = 'meta-llama/llama-4-scout-17b-16e-instruct'

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
product_manager = Agent(
    role="Product Manager",
    goal="Define a clear and user-focused product feature based on the requirement: {feature_request}",
    backstory="You're working on planning a new software feature: {feature_request}. "
              "You gather relevant requirements, define the problem the feature solves, "
              "identify user needs, and outline the core functionality. "
              "Your plan will be handed off to a Software Developer who will implement the feature.",
    allow_delegation=False,
    verbose=True
)

software_developer = Agent(
    role="Software Developer",
    goal="Implement the planned feature: {feature_request} based on the Product Manager’s outline",
    backstory="You're responsible for developing a software feature: {feature_request}. "
              "Your work is based on the Product Manager’s outline, including specifications, goals, and constraints. "
              "You are expected to write clean, maintainable, and efficient code that fulfills the stated requirements. "
              "You may include documentation and necessary tests.",
    allow_delegation=False,
    verbose=True
)

qa_engineer = Agent(
    role="QA Engineer",
    goal="Test and review the implementation of the feature: {feature_request} for quality and correctness",
    backstory="You are a QA Engineer responsible for validating the feature: {feature_request}, "
              "developed by the Software Developer. "
              "You verify that it meets all requirements defined by the Product Manager, "
              "functions correctly under various conditions, and does not introduce regressions. "
              "You ensure the software meets quality standards before release.",
    allow_delegation=False,
    verbose=True
)


## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

In [ ]:
define_feature = Task(
    description=(
        "1. Analyze the feature request: {feature_request} to determine its purpose and user value.\n"
        "2. Identify the target users, their goals, and the pain points this feature will address.\n"
        "3. Outline the core functionality, success criteria, and any constraints or edge cases.\n"
        "4. Provide user stories, acceptance criteria, and technical notes where appropriate."
    ),
    expected_output="A detailed feature specification document including user analysis, "
                    "functional requirements, user stories, and acceptance criteria.",
    agent=product_manager,
)

implement_feature = Task(
    description=(
        "1. Use the feature specification to implement the feature: {feature_request}.\n"
        "2. Write clean, well-documented, and maintainable code that adheres to best practices.\n"
        "3. Include unit tests and any necessary comments or documentation.\n"
        "4. Ensure the code meets the acceptance criteria and handles edge cases.\n"
        "5. Commit changes with clear messages describing what was done and why."
    ),
    expected_output="A functional code implementation of the feature with accompanying unit tests "
                    "and inline documentation, ready for code review.",
    agent=software_developer,
)

test_feature = Task(
    description=(
        "1. Review the implemented feature: {feature_request} for correctness, performance, and reliability.\n"
        "2. Validate that the feature meets all requirements and acceptance criteria.\n"
        "3. Perform exploratory and regression testing to ensure quality.\n"
        "4. Report any bugs, inconsistencies, or potential improvements."
    ),
    expected_output="A QA report detailing test results, issues found (if any), and validation that the feature "
                    "is ready for deployment or further refinement.",
    agent=qa_engineer
)


In [ ]:
crew = Crew(
    agents=[product_manager, software_developer, qa_engineer],
    tasks=[define_feature, implement_feature, test_feature],
    verbose=True
)

result = crew.kickoff(inputs={"feature_request": "Add JUnit support to the project"})

In [ ]:
from IPython.display import Markdown, display

# Assuming `result` is the output from `crew.kickoff(...)`
display(Markdown(result.raw))